# Adding Genre Data via Spotipy and Spotify API Access
- This will allow me to do more in depth analysis on my data

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Replace with your credentials
client_id = 'a7c9543950304b80910d8f05c4dc456f'
client_secret = '7fc00e590aff42c29beaad69223cd567'

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)



   ---------------------------------------- 0/2 [redis]
   ---------------------------------------- 0/2 [redis]
   ---------------------------------------- 0/2 [redis]
   ---------------------------------------- 0/2 [redis]
   ---------------------------------------- 0/2 [redis]
   ---------------------------------------- 0/2 [redis]
   -------------------- ------------------- 1/2 [spotipy]
   ---------------------------------------- 2/2 [spotipy]

Note: you may need to restart the kernel to use updated packages.
